In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch
import pandas as pd
from PIL import ImageDraw, ImageFont, Image
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
import torch.nn as nn
from transformers import ViTModel
from torchinfo import summary  # 
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import random
import time
from sklearn.utils.class_weight import compute_class_weight
from transformers import EfficientNetForImageClassification

In [2]:
DEVICE="cuda:0"
def setAllSeeds(seed):
  os.environ['MY_GLOBAL_SEED'] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
# setAllSeeds(42)

In [3]:
df = pd.read_csv("train.csv")
categories=df["Category"].unique()
print(categories)
category=categories[1]
df = df[df["Category"]==category]

['Men Tshirts' 'Sarees' 'Kurtis' 'Women Tshirts' 'Women Tops & Tunics']


In [4]:
df.isna().sum()

id              0
Category        0
len             0
attr_1      10461
attr_2        667
attr_3       2485
attr_4        450
attr_5        697
attr_6      13336
attr_7       9450
attr_8       1881
attr_9       4043
attr_10       528
dtype: int64

In [5]:
attr_i=1
removing_attri=[]
df.iloc[:,3:2+attr_i]=np.nan
df.iloc[:,3+attr_i:]=np.nan


In [6]:
print(df.value_counts())

Series([], Name: count, dtype: int64)


In [7]:
delCol = []
idxCol = []
trackNum = []
weights=[]
m=nn.Softmax(dim=1)
l=[]
for i in range(1,11):
    uniName = df["attr_"+str(i)].dropna().unique()
    print(uniName)
    if(len(uniName)==0):
        delCol.append("attr_"+str(i))
    else:
        idxCol.append("attr_"+str(i))
        l.append('attr_'+str(i))
        trackNum.append(len(uniName))
        class_weights=compute_class_weight(class_weight="balanced",classes=uniName,y=df["attr_"+str(i)].dropna())
        weights.append(torch.tensor([1]*len(uniName),dtype=torch.float32).to(DEVICE))


['same as saree' 'solid' 'same as border' 'default']
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [8]:
df = df.drop(delCol,axis=1)
df.shape

(18346, 4)

In [9]:
df.isna().sum()

id              0
Category        0
len             0
attr_1      10461
dtype: int64

In [10]:
# df.dropna(inplace=True)

In [11]:
id2label={}
label2id={}
attrs={}
total_attr=len(df.columns)
for i in range(3,total_attr):
    labels=df[df.columns[i]].dropna().unique()
    # print(df.columns[i],labels)
    id2label[i-3]={k:labels[k] for k in range(len(labels))}
    label2id[i-3]={labels[k]:k for k in range(len(labels))}
    attrs[i-3]=df.columns[i]
print(id2label)
print(label2id)
print(attrs)

{0: {0: 'same as saree', 1: 'solid', 2: 'same as border', 3: 'default'}}
{0: {'same as saree': 0, 'solid': 1, 'same as border': 2, 'default': 3}}
{0: 'attr_1'}


In [12]:
def categorize(example):
    for i in attrs:
        # print(example[attrs[i]],type(example[attrs[i]]),pd.isna(example[attrs[i]]))
        if not pd.isna(example[attrs[i]]):
            example[attrs[i]]=label2id[i][example[attrs[i]]]
        else:
            example[attrs[i]]=-100
    return example
df=df.apply(categorize,axis=1)
df.head()

,id,Category,len,attr_1
7267,7432,Sarees,10,0
7268,7433,Sarees,10,-100
7269,7434,Sarees,10,-100
7270,7435,Sarees,10,0
7271,7436,Sarees,10,1


In [13]:
df.isna().sum()

id          0
Category    0
len         0
attr_1      0
dtype: int64

In [14]:
val_df = df

In [15]:
from sklearn.model_selection import train_test_split


train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
min_count = train_df[f'attr_{attr_i}'].value_counts().min()
print(f"The minimum count across all categories is: {min_count}")
df_sampled = train_df.groupby(f'attr_{attr_i}').sample(n=min_count, random_state=42)
train_df = df_sampled

The minimum count across all categories is: 177


In [16]:
for i in l:
    print(train_df[i].value_counts())

attr_1
-100    177
 0      177
 1      177
 2      177
 3      177
Name: count, dtype: int64


In [17]:
# weights[4]=torch.tensor([1,20],dtype=torch.float32).to(DEVICE)
# weights[9]=torch.tensor([1,1,1,1,3,2],dtype=torch.float32).to(DEVICE)
# weights[7]=torch.tensor([1,10],dtype=torch.float32).to(DEVICE)

In [18]:
# for i in l:
#     print(i)
#     for j in set(train_df[i].unique()):
#         x=train_df[i][train_df[i]==j]
#         if x.count()<300 and j!=-100:
#             y=train_df[train_df[i]==j]
#             y.iloc[:,3:]=-100
#             y[i]=j
#             print(y)
#             train_df=pd.concat([train_df,y,y]).reset_index(drop=True)
# for i in l:
#     print(train_df[i].value_counts())

In [19]:
train_df.head()

,id,Category,len,attr_1
22095,22260,Sarees,10,-100
7608,7773,Sarees,10,-100
23614,23779,Sarees,10,-100
14836,15001,Sarees,10,-100
23116,23281,Sarees,10,-100


In [20]:
print(train_df[f'attr_{attr_i}'].value_counts())
print(val_df[f'attr_{attr_i}'].value_counts())

attr_1
-100    177
 0      177
 1      177
 2      177
 3      177
Name: count, dtype: int64
attr_1
-100    1046
 0       539
 2       155
 3        84
 1        11
Name: count, dtype: int64


In [21]:
from transformers import AutoImageProcessor
# model_name = 'google/vit-base-patch16-224'
# model_name=f"vit5/{category}/final"
model_name = 'google/efficientnet-b0'
# model_name = 'vit6/'+category+'/final'
# model_name="google/vit-large-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)

In [22]:
class CustomFashionManager(Dataset):
    def __init__(self,csv_file, root_dir="./",transforms =None):
        self.fashionItems = csv_file
        self.root_dir = root_dir
        self.transforms = transforms
    
    def __len__(self):
        return len(self.fashionItems)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,f"{self.fashionItems.iloc[idx, 0]:06d}"+'.jpg')
        image = Image.open(img_name)
        attributes = self.fashionItems.iloc[idx, 3:]
        attributes = np.array(attributes)
        attributes = attributes.astype('float')
        
        inputs=processor(image, return_tensors='pt')
        inputs['labels']=torch.tensor(attributes, dtype=torch.long)
        # if self.transforms:
        #     inputs['pixel_values'] = self.transforms(inputs['pixel_values'])
        

        return inputs


In [23]:
from torchvision import transforms
transformations=transforms.Compose([
    transforms.RandomRotation(degrees=90),
    transforms.GaussianBlur(kernel_size=(9, 9), sigma=(0.1, 2.0)),
])
transformations=None
train_fashion_data = CustomFashionManager(csv_file=train_df,
                                    root_dir='train_images')
val_fashion_data = CustomFashionManager(csv_file=val_df,
                                    root_dir='train_images')
test_fashion_data = CustomFashionManager(csv_file=df,root_dir='train_images')
        

In [24]:
import sys
from typing import List
from transformers import (
    ViTConfig,ViTPreTrainedModel,EfficientNetConfig,EfficientNetPreTrainedModel,
    EfficientNetModel)


class CustomConfig(EfficientNetConfig):
    def __init__(self,num_classes_per_label:List[int]=[1],**kwargs):
        super().__init__(**kwargs)
        self.num_classes_per_label = num_classes_per_label
        # self.attn_implementation="sdpa"
        # self.hidden_size_2 = 382
        # self.attention_probs_dropout_prob=0.1
        # self.hidden_dropout_prob=0.1

            
class MultiLabelMultiClassViT(EfficientNetPreTrainedModel):
    config_class=CustomConfig
    def __init__(self, config: CustomConfig) -> None:
        super().__init__(config)

        self.efficientnet = EfficientNetModel(config)
        self.classifiers = nn.ModuleList([
            nn.Sequential(nn.Dropout(0.2),
            # nn.Linear(config.hidden_size, 32),
            # nn.ReLU(),
            # nn.Dropout(0.2),
            nn.Linear(config.hidden_dim, num_classes))
            # nn.Linear(49, num_classes)) 
            for num_classes in config.num_classes_per_label
        ])
        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, pixel_values,labels=None):
        # outputs = self.efficientnet(pixel_values).last_hidden_state[:, 0, :]  # CLS token representation
        # outputs = outputs.reshape(outputs.shape[0],-1)
        outputs=self.efficientnet(pixel_values).pooler_output
        logits = [classifier(outputs) for classifier in self.classifiers]
        if labels is not None:
            loss=0
            # loss_weights=np.array([1,1,1,1,1,1,1,5])
            # loss_weights=loss_weights/loss_weights.sum()
            for i in range(len(logits)):
                target=labels[:,i]
                loss += torch.nn.functional.cross_entropy(logits[i], target,ignore_index=-100)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

    
# Example usage
num_labels = len(trackNum)  # For example, 5 different labels



In [25]:
config=EfficientNetConfig.from_pretrained(model_name)
config=CustomConfig(num_classes_per_label=trackNum,**config.to_dict())
model = MultiLabelMultiClassViT.from_pretrained(model_name, config=config)

# model=MultiLabelMultiClassViT.from_pretrained(model_name)
# for param in model.vit.parameters():
#     param.requires_grad = False
# model.vit.pooler.parameters().__next__().requires_grad=True
print(summary(model))

Some weights of MultiLabelMultiClassViT were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized: ['classifiers.0.1.bias', 'classifiers.0.1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                            Param #
MultiLabelMultiClassViT                                           --
├─EfficientNetModel: 1-1                                          --
│    └─EfficientNetEmbeddings: 2-1                                --
│    │    └─ZeroPad2d: 3-1                                        --
│    │    └─Conv2d: 3-2                                           864
│    │    └─BatchNorm2d: 3-3                                      64
│    │    └─SiLU: 3-4                                             --
│    └─EfficientNetEncoder: 2-2                                   --
│    │    └─ModuleList: 3-5                                       3,594,460
│    │    └─Conv2d: 3-6                                           409,600
│    │    └─BatchNorm2d: 3-7                                      2,560
│    │    └─SiLU: 3-8                                             --
│    └─AvgPool2d: 2-3                                             --
├─ModuleList:

In [26]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import classification_report,f1_score
batch_size = 32
def collate_fn(batch):
    return {
        'pixel_values': torch.cat([x['pixel_values'] for x in batch],dim=0),
        'labels': torch.stack([x['labels'] for x in batch])
    }

def compute_metrics(pred):
    logits = pred.predictions
    labels=pred.label_ids
    probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    probs=probs.T
    f1s=[]
    for i in range(labels.shape[1]):
        non_padding_indices = [j for j, label in enumerate(labels[:,i]) if label != -100]
        labels_ = [labels[j,i] for j in non_padding_indices]
        probs_ = [probs[j,i] for j in non_padding_indices]
        micro=f1_score(labels_,probs_,average='micro')
        macro=f1_score(labels_,probs_,average='macro')
        print(f"attr_{i} f1 score: {macro}")
        # print(classification_report(labels_,probs_))
        score=2*(micro*macro)/(micro+macro)
        f1s.append(score)
    print()
    print()
    return {'score': sum(f1s)/len(f1s)}


    # logits = pred.predictions
    # labels=pred.label_ids
    # probs = np.stack([np.argmax(logit,axis=1) for logit in logits])
    # probs=probs.T
    # labels=labels.flatten()
    # probs=probs.flatten()
    # non_padding_indices = [i for i, label in enumerate(labels) if label != -100]
    # labels = [labels[i] for i in non_padding_indices]
    # probs = [probs[i] for i in non_padding_indices]
    # print(classification_report(labels,probs))
    # report=classification_report(labels,probs,output_dict=True)
    # return {'accuracy': report['accuracy'],"macro avg f1":report['macro avg']['f1-score']}

training_args = TrainingArguments(
  output_dir="./vit6/"+category,
  per_device_train_batch_size=128,
  per_device_eval_batch_size=128,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  logging_strategy="epoch",
  num_train_epochs=1,
  bf16=True,
  learning_rate=2e-4,
#   dataloader_num_workers=20,lr
  save_total_limit=1,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  metric_for_best_model="score"
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_fashion_data,
    eval_dataset=val_fashion_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train()
trainer.save_model(f"./vit6/{category}/final",safe_serialization=False)

[2024-11-16 15:10:28,840] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/aseems/anaconda3/envs/mhcp4/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Score
1,1.205500,1.213278,0.458246


attr_0 f1 score: 0.41303227187448976




TypeError: save_model() got an unexpected keyword argument 'safe_serialization'

In [ ]:
trainer.evaluate(test_fashion_data)
#1-54(imbalance),2-65,3-85,5-56,6-70,7-52(imbalance),8-51(imbalance),9-60(imbalance),10-49(imbalance)
#4-30,

attr_0 f1 score: 0.5430645408423548




{'eval_loss': 0.8007969856262207,
 'eval_score': 0.5964635895277907,
 'eval_runtime': 89.996,
 'eval_samples_per_second': 203.853,
 'eval_steps_per_second': 1.6,
 'epoch': 40.0}